In [1]:
!python --version

Python 3.10.12


In [2]:
# Install the requirements.
!pip install -q transformers einops accelerate langchain bitsandbytes
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.9 MB/s eta 0:00:00


In [3]:
!huggingface-cli login
# You need to have the account to login.
# Use your huggingface token to enter.


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGrained).
The token `llama2` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate w

Creating the Pipeline

In [4]:
"""
The code initializes a text-generation pipeline using the Hugging Face Transformers library
with the Llama-2-7b-chat model, configuring it for efficient execution on available hardware
with optimized settings for sampling, maximum sequence length, and other parameters.
"""
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer
import transformers
import torch
import time

model = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model)

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    # trust_remote=True, # removing this because getting error.
    device_map="auto",
    max_length=1000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

Sample Input

In [5]:
# Use the following code for running an example input/output.
"""
sample_model_input = tokenizer.apply_chat_template([{'role': 'user', 'content': "Who is a gladiator?"}], tokenize=False, add_generation_prompt=True)
generation_output = pipeline(sample_model_input)

# Extract the generated text from the output dictionary
generated_text = generation_output[0]['generated_text']

# Decode the generated text
decoded_response = tokenizer.decode(tokenizer.encode(generated_text))

print(decoded_response)
"""

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


<s><s> [INST] Who is a gladiator? [/INST]  A gladiator was a professional fighter in ancient Rome who participated in the gladiatorial games, which were bloody and often deadly entertainment events held in large arenas.

Gladiators were typically slaves, prisoners of war, or criminals who were forced to fight each other to the death as part of a spectacle. They were trained in various combat styles, including swordsmanship, spear throwing, and hand-to-hand combat, and were often pitted against each other in battles that could last for hours or even days.

Gladiators were considered to be the elite warriors of ancient Rome, and their bravery and skill were revered by the Roman people. They were often rewarded with gifts, money, and freedom if they emerged victorious in a fight, and their exploits were celebrated in literature and art.

Some of the most famous gladiators in ancient Rome include:

* Spartacus: A Thracian gladiator who led a major slave uprising against the Roman Empire in

Create the LLM object

In [6]:
# create the llm
llm = HuggingFacePipeline(pipeline=pipeline, model_kwargs = {'temperature':0.3})

<ipython-input-6-357025261a9e>:2: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipeline, model_kwargs = {'temperature':0.3})


Build the template and prompt

In [7]:
from langchain import PromptTemplate, LLMChain

template = """
You are an experienced old English to modern English translator. Do the following:
1) Translate the given Shakespearean text to modern English. An example is given below. Do not add or edit new information in translation.
2) All the dialogues need to be translated. Do not leave any.
3) Do not add any end notes or comments after translation.
4) Keep all the stage direction notes like titles, content details, [_Exeunt._], [_Exit_], and anything in between [_ and _] as such.

Example)
CALPHURNIA, wife to Caesar
PORTIA, wife to Brutus
SCENE: Rome, the conspirators’ camp near Sardis, and the plains of Philippi.
ACT III
Enter Flavius, Marullus and a throng of Citizens.

FLAVIUS.
Hence! home, you idle creatures, get you home.
Is this a holiday? What, know you not,
Being mechanical, you ought not walk
Upon a labouring day without the sign
Of your profession? Speak, what trade art thou?

MARULLUS.
Did from the flames of Troy upon his shoulder
The old Anchises bear, so from the waves of Tiber
Did I the tired Caesar. And this man
Is now become a god; and Cassius is
A wretched creature, and must bend his body,
If Caesar carelessly but nod on him.
[_Exeunt._]

Translation)
CALPHURNIA, wife to Caesar
PORTIA, wife to Brutus
SCENE: Rome, the conspirators’ camp near Sardis, and the plains of Philippi.
ACT III
[Enter Flavius, Marullus, and a crowd of citizens.]

FLAVIUS.
Go home, you lazy people, get back to your houses!
Is this a holiday? Don’t you know
That as working-class folks, you shouldn’t be out
On a workday without tools to show your trade?
Tell me, what’s your job?

MARULLUS.
He carried his father Anchises on his shoulders out of the flames of Troy. In the same way, I carried the exhausted Caesar from the waves of the Tiber River. And now this man has become a god, while I, Cassius, am a miserable creature who must bow down whenever Caesar gives even the slightest nod.
[_Exeunt._]

------------------
INPUT TEXT
------------------

{text}
------------------------------
TRANSLATION
**********
"""
prompt = PromptTemplate(template=template, input_variables=["text"])

Preprocessing

In [ ]:
file_path = "./pp_JulicusCaesar.txt"
def load_text_in_parts_with_indentation(file_path, max_words=100):
    """
    Load the contents of a text file in parts, preserving indentations,
    where each part contains a maximum of max_words.

    Args:
        file_path (str): Path to the text file.
        max_words (int): Maximum number of words per part.

    Returns:
        list: A list of strings, each containing up to max_words words, with indentations preserved.
    """
    parts = []
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()  # Read the entire content

        # Preserve indentation by splitting into paragraphs
        paragraphs = content.splitlines(keepends=True)

        current_part = []
        current_word_count = 0

        for paragraph in paragraphs:
            words_in_paragraph = paragraph.split()  # Split the paragraph into words
            paragraph_word_count = len(words_in_paragraph)

            if current_word_count + paragraph_word_count > max_words:
                # If adding this paragraph exceeds max_words, save the current part
                parts.append(''.join(current_part))
                current_part = []  # Start a new part
                current_word_count = 0

            # Add the paragraph to the current part
            current_part.append(paragraph)
            current_word_count += paragraph_word_count

        # Add the remaining part if any
        if current_part:
            parts.append(''.join(current_part))

    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found.")
    except Exception as e:
        print(f"An error occurred: {e}")

    return parts

load_text_in_parts_with_indentation(file_path = file_path)

['\ufeffTHE TRAGEDY OF\nJULIUS CAESAR\n\n\nby William Shakespeare\n\n\n\n\nContents\n\n ACT I\n Scene I. Rome. A street\n Scene II. The same. A public place\n Scene III. The same. A street\n\n ACT II\n Scene I. Rome. Brutus’ orchard\n Scene II. A room in Caesar’s palace\n Scene III. A street near the Capitol\n Scene IV. Another part of the same street, before the house of Brutus\n\n ACT III\n Scene I. Rome. Before the Capitol; the Senate sitting\n Scene II. The same. The Forum\n Scene III. The same. A street\n\n ACT IV\n Scene I. A room in Antony’s house\n',
 ' Scene II. Before Brutus’ tent, in the camp near Sardis\n Scene III. Within the tent of Brutus\n\n ACT V\n Scene I. The plains of Philippi\n Scene II. The same. The field of battle\n Scene III. Another part of the field\n Scene IV. Another part of the field\n Scene V. Another part of the field\n\n\n\n\n Dramatis Personæ\n\n\nJULIUS CAESAR\nOCTAVIUS CAESAR, Triumvir after his death.\nMARCUS ANTONIUS,    ”    ”     ”\nM. AEMILIUS L

In [9]:
# Use the code below if you want to know how the model tokenizes your input.
# from transformers import LlamaTokenizer

# # Load the tokenizer for Llama 2
# # tokenizer = LlamaTokenizer.from_pretrained("meta-llama/Llama-2-7b")

# # Example text
# example_text = load_text_in_parts_with_indentation(file_path = file_path)[10]

# # Tokenize the text
# tokens = tokenizer.encode(example_text, add_special_tokens=True)
# # print(tokens)

# # Count the tokens
# token_count = len(tokens)

# print(token_count)

Sample output from the translation model

In [10]:
llm_chain = LLMChain(prompt=prompt, llm=llm)
text = """
CALPHURNIA.
When beggars die, there are no comets seen;
The heavens themselves blaze forth the death of princes.
"""
print(llm_chain.run(text=text, max_new_tokens=1200))  # Adjust 1200 as needed

<ipython-input-10-42587a31abe5>:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(prompt=prompt, llm=llm)
<ipython-input-10-42587a31abe5>:7: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(llm_chain.run(text=text, max_new_tokens=1200))  # Adjust 1200 as needed



You are an experienced old English to modern English translator. Do the following:
1) Translate the given Shakespearean text to modern English. An example is given below. Do not add or edit new information in translation.
2) All the dialogues need to be translated. Do not leave any.
3) Do not add any end notes or comments after translation.
4) Keep all the stage direction notes like titles, content details, [_Exeunt._], [_Exit_], and anything in between [_ and _] as such.

Example)
CALPHURNIA, wife to Caesar
PORTIA, wife to Brutus
SCENE: Rome, the conspirators’ camp near Sardis, and the plains of Philippi.
ACT III
Enter Flavius, Marullus and a throng of Citizens.

FLAVIUS.
Hence! home, you idle creatures, get you home.
Is this a holiday? What, know you not,
Being mechanical, you ought not walk
Upon a labouring day without the sign
Of your profession? Speak, what trade art thou?

MARULLUS.
Did from the flames of Troy upon his shoulder
The old Anchises bear, so from the waves of Tiber
D

In [11]:
# start_time = time.time()
# # Instead of using max_length, use max_new_tokens
# print(llm_chain.run(text=load_text_in_parts_with_indentation(file_path=file_path)[1], max_new_tokens=1200))  # Adjust 200 as needed
# end_time = time.time()
# print(f"Time taken: {end_time - start_time} seconds or {(end_time - start_time) / 60} minutes")

Saving

In [ ]:
import os

def save_output_to_file(llm_chain, file_path, output_file_path):
    """
    Runs the LLM chain on input text loaded from the file,
    and saves the output to a new text file.

    Parameters:
        llm_chain: The LLM chain object.
        file_path: Path to the input text file.
        output_file_path: Path to save the output text file.
    """
    parts = load_text_in_parts_with_indentation(file_path)
    parts_len = len(load_text_in_parts_with_indentation(file_path))

    # Assuming `load_text_in_parts_with_indentation` is a function that reads and processes the input text.
    for i in range(1, parts_len):
      input_text = parts[i]
      # Run the LLM chain to generate the output.
      output = llm_chain.run(text=input_text, max_new_tokens=1200)
      print(f"Output for part {i}/{parts_len} generated ")
      # Construct the output file name based on the part number.
      part_file_name = f"V110_Julius_out_{i}.txt" #V110 is just the version name. 
      part_file_path = os.path.join(output_file_path, part_file_name)

      # Save the output to the specified file.
      with open(part_file_path, 'w', encoding='utf-8') as file:
          file.write(output)
          print(f"Output for part {i} saved.")

    print(f"All Output texts successfully saved to {output_file_path}")


# Example Usage
file_path = "./pp_JulicusCaesar.txt" # Path to your input text file.
output_file_path = "./preprocessed_data/"  # Path to save the output.
# Make sure the output directory exists.
os.makedirs(output_file_path, exist_ok=True)
save_output_to_file(llm_chain, file_path, output_file_path)
